In [1]:
"""
Funktionsgenerator für Raspberry Pi mit MCC DAQ HAT
"""

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# Abtastrate und andere Konstanten
ABTASTRATE = 10000  # Hz
MAX_AMPLITUDE = 5.0  # Maximale Amplitude in Volt

class Signalgenerator:
    """
    Klasse zur Erzeugung verschiedener Signalformen.
    """
    
    def __init__(self, abtastrate=ABTASTRATE):
        """
        Initialisiert den Signalgenerator.
        
        Args:
            abtastrate (int): Abtastrate in Hz
        """
        self.abtastrate = abtastrate
        self.dac_kanal = 0  # DAC-Kanal (kann angepasst werden)
    
    def sinus_welle(self, frequenz, amplitude, offset=0, zeit_s=0.1):
        """
        Erzeugt eine Sinuswelle.
        
        Args:
            frequenz (float): Frequenz in Hz
            amplitude (float): Amplitude in Volt
            offset (float): DC-Offset in Volt
            zeit_s (float): Signaldauer in Sekunden
            
        Returns:
            tuple: (zeit, signal)
        """
        t = np.linspace(0, zeit_s, int(zeit_s * self.abtastrate), endpoint=False)
        signal = amplitude * np.sin(2 * np.pi * frequenz * t) + offset
        return t, signal
    
    def rechteck_welle(self, frequenz, amplitude, offset=0, zeit_s=0.1, tastgrad=0.5):
        """
        Erzeugt eine Rechteckwelle.
        
        Args:
            frequenz (float): Frequenz in Hz
            amplitude (float): Amplitude in Volt
            offset (float): DC-Offset in Volt
            zeit_s (float): Signaldauer in Sekunden
            tastgrad (float): Tastgrad (0-1)
            
        Returns:
            tuple: (zeit, signal)
        """
        t = np.linspace(0, zeit_s, int(zeit_s * self.abtastrate), endpoint=False)
        signal = amplitude * (((t * frequenz) % 1) < tastgrad) + offset
        return t, signal
    
    def dreieck_welle(self, frequenz, amplitude, offset=0, zeit_s=0.1):
        """
        Erzeugt eine Dreieckwelle.
        
        Args:
            frequenz (float): Frequenz in Hz
            amplitude (float): Amplitude in Volt
            offset (float): DC-Offset in Volt
            zeit_s (float): Signaldauer in Sekunden
            
        Returns:
            tuple: (zeit, signal)
        """
        t = np.linspace(0, zeit_s, int(zeit_s * self.abtastrate), endpoint=False)
        signal = amplitude * (2 * np.abs(2 * ((t * frequenz) % 1) - 1) - 1) + offset
        return t, signal
    
    def saegezahn_welle(self, frequenz, amplitude, offset=0, zeit_s=0.1, anstieg=True):
        """
        Erzeugt eine Sägezahnwelle.
        
        Args:
            frequenz (float): Frequenz in Hz
            amplitude (float): Amplitude in Volt
            offset (float): DC-Offset in Volt
            zeit_s (float): Signaldauer in Sekunden
            anstieg (bool): True für ansteigende, False für absteigende Flanke
            
        Returns:
            tuple: (zeit, signal)
        """
        t = np.linspace(0, zeit_s, int(zeit_s * self.abtastrate), endpoint=False)
        if anstieg:
            signal = amplitude * (2 * ((t * frequenz) % 1) - 1) + offset
        else:
            signal = amplitude * (1 - 2 * ((t * frequenz) % 1)) + offset
        return t, signal
    
    def rauschen(self, amplitude, offset=0, zeit_s=0.1):
        """
        Erzeugt weißes Rauschen.
        
        Args:
            amplitude (float): Amplitude in Volt
            offset (float): DC-Offset in Volt
            zeit_s (float): Signaldauer in Sekunden
            
        Returns:
            tuple: (zeit, signal)
        """
        t = np.linspace(0, zeit_s, int(zeit_s * self.abtastrate), endpoint=False)
        signal = amplitude * np.random.normal(0, 0.5, len(t)) + offset
        return t, signal
    
    def signal_zum_dac_senden(self, signal_daten):
        """
        Sendet das Signal zum DAC.
        
        Args:
            signal_daten (np.array): Signaldaten
            
        Hinweis: Diese Funktion muss an die tatsächliche Hardware angepasst werden.
        """
        # Hier den Code zum Senden der Daten an den DAC einfügen
        # Beispiel für MCC DAQ (muss angepasst werden):
        """
        from mcculw import ul
        from mcculw.enums import ULRange
        
        for wert in signal_daten:
            # Sende Wert an DAC (Kanal, Bereich, Wert)
            ul.v_out(0, self.dac_kanal, ULRange.BIP5VOLTS, wert)
            time.sleep(1/self.abtastrate)
        """
        # Simulierte Implementierung (Platzhalter):
        print(f"Signal mit {len(signal_daten)} Datenpunkten würde an DAC gesendet werden")
        
# Erstellen der GUI
class FunktionsgeneratorGUI:
    """
    GUI für den Funktionsgenerator.
    """
    
    def __init__(self):
        """
        Initialisiert die GUI.
        """
        self.generator = Signalgenerator()
        self.ausgabe_aktiv = False
        self.ausgabe_thread = None
        self.setup_gui()
        
    def setup_gui(self):
        """
        Erstellt die GUI-Elemente.
        """
        # Wellenform-Auswahl
        self.wellenform_dropdown = widgets.Dropdown(
            options=[
                ('Sinus', 'sinus'),
                ('Rechteck', 'rechteck'),
                ('Dreieck', 'dreieck'),
                ('Sägezahn', 'saegezahn'),
                ('Rauschen', 'rauschen')
            ],
            value='sinus',
            description='Wellenform:',
            style={'description_width': 'initial'}
        )
        
        # Frequenz-Regler
        self.frequenz_slider = widgets.FloatSlider(
            value=100,
            min=0.1,
            max=1000,
            step=0.1,
            description='Frequenz (Hz):',
            style={'description_width': 'initial'}
        )
        
        # Logarithmischer Frequenz-Regler
        self.frequenz_log_slider = widgets.FloatLogSlider(
            value=100,
            base=10,
            min=0,  # 10^0 = 1 Hz
            max=3,  # 10^3 = 1000 Hz
            step=0.01,
            description='Frequenz (log):',
            style={'description_width': 'initial'}
        )
        
        # Verknüpfung der Frequenzregler
        widgets.jslink((self.frequenz_slider, 'value'), (self.frequenz_log_slider, 'value'))
        
        # Amplitude-Regler
        self.amplitude_slider = widgets.FloatSlider(
            value=1.0,
            min=0.0,
            max=MAX_AMPLITUDE,
            step=0.1,
            description='Amplitude (V):',
            style={'description_width': 'initial'}
        )
        
        # Offset-Regler
        self.offset_slider = widgets.FloatSlider(
            value=0.0,
            min=-MAX_AMPLITUDE,
            max=MAX_AMPLITUDE,
            step=0.1,
            description='DC-Offset (V):',
            style={'description_width': 'initial'}
        )
        
        # Tastgrad-Regler (für Rechteck)
        self.tastgrad_slider = widgets.FloatSlider(
            value=0.5,
            min=0.1,
            max=0.9,
            step=0.1,
            description='Tastgrad:',
            style={'description_width': 'initial'}
        )
        
        # Anstieg/Abstieg-Schalter (für Sägezahn)
        self.anstieg_switch = widgets.Checkbox(
            value=True,
            description='Ansteigende Flanke',
            style={'description_width': 'initial'}
        )
        
        # Start/Stop-Button
        self.start_button = widgets.Button(
            description='Start Output',
            button_style='success',
            tooltip='Starten/Stoppen der Signalausgabe'
        )
        self.start_button.on_click(self.toggle_ausgabe)
        
        # Einmalige Ausgabe-Button
        self.ausgabe_button = widgets.Button(
            description='Signal anzeigen',
            button_style='info',
            tooltip='Zeigt das aktuelle Signal an'
        )
        self.ausgabe_button.on_click(self.signal_anzeigen)
        
        # Layout-Container erstellen
        allgemeine_parameter = widgets.VBox([
            self.wellenform_dropdown,
            self.frequenz_slider,
            self.frequenz_log_slider,
            self.amplitude_slider, 
            self.offset_slider
        ])
        
        spezielle_parameter = widgets.VBox([
            widgets.Label('Spezielle Parameter:'),
            self.tastgrad_slider,
            self.anstieg_switch
        ])
        
        buttons = widgets.HBox([self.ausgabe_button, self.start_button])
        
        # Hauptlayout
        self.haupt_layout = widgets.VBox([
            widgets.HBox([allgemeine_parameter, spezielle_parameter]),
            buttons
        ])
        
        # Bild-Container
        self.output = widgets.Output()
        
        # GUI anzeigen
        display(self.haupt_layout)
        display(self.output)
        
        # Verbinde Änderungsereignisse
        self.wellenform_dropdown.observe(self.update_anzeige, names='value')
        self.frequenz_slider.observe(self.update_anzeige, names='value')
        self.amplitude_slider.observe(self.update_anzeige, names='value')
        self.offset_slider.observe(self.update_anzeige, names='value')
        self.tastgrad_slider.observe(self.update_anzeige, names='value')
        self.anstieg_switch.observe(self.update_anzeige, names='value')
        
        # Initialisieren der Anzeige
        self.update_anzeige(None)
        
    def update_anzeige(self, change):
        """
        Aktualisiert die Anzeige der Wellenform.
        """
        wellenform = self.wellenform_dropdown.value
        frequenz = self.frequenz_slider.value
        amplitude = self.amplitude_slider.value
        offset = self.offset_slider.value
        
        # Aktiviere/Deaktiviere Parameter je nach Wellenform
        if wellenform == 'rechteck':
            self.tastgrad_slider.disabled = False
            self.anstieg_switch.disabled = True
        elif wellenform == 'saegezahn':
            self.tastgrad_slider.disabled = True
            self.anstieg_switch.disabled = False
        else:
            self.tastgrad_slider.disabled = True
            self.anstieg_switch.disabled = True
            
        # Deaktiviere Frequenzregler für Rauschen
        self.frequenz_slider.disabled = (wellenform == 'rauschen')
        self.frequenz_log_slider.disabled = (wellenform == 'rauschen')
        
        # Nur aktualisieren, wenn ausgabe_button geklickt wurde
        if not hasattr(self, 'letzte_anzeige_aktualisiert') or self.letzte_anzeige_aktualisiert:
            self.letzte_anzeige_aktualisiert = False
    
    def signal_anzeigen(self, b):
        """
        Zeigt das ausgewählte Signal an.
        """
        with self.output:
            clear_output(wait=True)
            
            # Parameter auslesen
            wellenform = self.wellenform_dropdown.value
            frequenz = self.frequenz_slider.value
            amplitude = self.amplitude_slider.value
            offset = self.offset_slider.value
            tastgrad = self.tastgrad_slider.value
            anstieg = self.anstieg_switch.value
            
            # Signal generieren
            if wellenform == 'sinus':
                t, signal = self.generator.sinus_welle(frequenz, amplitude, offset)
                titel = f"Sinuswelle: {frequenz} Hz, {amplitude} V"
            elif wellenform == 'rechteck':
                t, signal = self.generator.rechteck_welle(frequenz, amplitude, offset, tastgrad=tastgrad)
                titel = f"Rechteckwelle: {frequenz} Hz, {amplitude} V, Tastgrad {tastgrad}"
            elif wellenform == 'dreieck':
                t, signal = self.generator.dreieck_welle(frequenz, amplitude, offset)
                titel = f"Dreieckwelle: {frequenz} Hz, {amplitude} V"
            elif wellenform == 'saegezahn':
                t, signal = self.generator.saegezahn_welle(frequenz, amplitude, offset, anstieg=anstieg)
                titel = f"Sägezahnwelle: {frequenz} Hz, {amplitude} V, Anstieg: {anstieg}"
            elif wellenform == 'rauschen':
                t, signal = self.generator.rauschen(amplitude, offset)
                titel = f"Rauschen: {amplitude} V"
            
            # Plot erstellen
            plt.figure(figsize=(10, 6))
            plt.plot(t, signal)
            plt.title(titel)
            plt.xlabel('Zeit (s)')
            plt.ylabel('Amplitude (V)')
            plt.grid(True)
            
            # Amplitude + Offset-Grenzen anzeigen
            plt.axhline(y=offset + amplitude, color='r', linestyle='--', alpha=0.3)
            plt.axhline(y=offset - amplitude, color='r', linestyle='--', alpha=0.3)
            plt.axhline(y=offset, color='g', linestyle='--', alpha=0.3)
            
            # Y-Achsen-Grenzen setzen
            plt.ylim(min(offset - amplitude - 0.5, -MAX_AMPLITUDE-0.5), max(offset + amplitude + 0.5, MAX_AMPLITUDE+0.5))
            
            plt.show()
            
            # Statistik ausgeben
            print(f"Signaldaten: Min={signal.min():.2f}V, Max={signal.max():.2f}V, Mittelwert={signal.mean():.2f}V")
            print(f"Abtastrate: {self.generator.abtastrate} Hz, Datenpunkte: {len(signal)}")
            
            # Aktualisierungsflag setzen
            self.letzte_anzeige_aktualisiert = True
            
    def toggle_ausgabe(self, b):
        """
        Startet oder stoppt die kontinuierliche Signalausgabe.
        """
        self.ausgabe_aktiv = not self.ausgabe_aktiv
        
        if self.ausgabe_aktiv:
            self.start_button.description = 'Stop Output'
            self.start_button.button_style = 'danger'
            
            # Parameter auslesen
            wellenform = self.wellenform_dropdown.value
            frequenz = self.frequenz_slider.value
            amplitude = self.amplitude_slider.value
            offset = self.offset_slider.value
            tastgrad = self.tastgrad_slider.value
            anstieg = self.anstieg_switch.value
            
            # Signal generieren
            if wellenform == 'sinus':
                _, signal = self.generator.sinus_welle(frequenz, amplitude, offset)
            elif wellenform == 'rechteck':
                _, signal = self.generator.rechteck_welle(frequenz, amplitude, offset, tastgrad=tastgrad)
            elif wellenform == 'dreieck':
                _, signal = self.generator.dreieck_welle(frequenz, amplitude, offset)
            elif wellenform == 'saegezahn':
                _, signal = self.generator.saegezahn_welle(frequenz, amplitude, offset, anstieg=anstieg)
            elif wellenform == 'rauschen':
                _, signal = self.generator.rauschen(amplitude, offset)
            
            # Simulierte Ausgabe (hier würde die tatsächliche DAC-Ausgabe erfolgen)
            self.generator.signal_zum_dac_senden(signal)
            print(f"Kontinuierliche Ausgabe gestartet: {wellenform.capitalize()}-Welle")
            
        else:
            self.start_button.description = 'Start Output'
            self.start_button.button_style = 'success'
            print("Ausgabe gestoppt")
            
# Funktionsgenerator starten
generator_gui = FunktionsgeneratorGUI()


Output()